# Goal

Show all London subway stations

## Imports and generic utility methods

In [1]:
import duckdb
import geopandas as gpd
from shapely import wkt

In [2]:
def duckdb_connection():
    conn = duckdb.connect()
    conn.install_extension('spatial')
    conn.load_extension('spatial')
    return conn

In [11]:
def load_area(conn, overture_local_base, overture_theme, overture_type, bbox):
    (minx,miny,maxx,maxy) = bbox
    path = f"{overture_local_base}/theme={overture_theme}/type={overture_type}/*"
    query = f"""
    SELECT * EXCLUDE(geometry), ST_AsText(geometry) AS geometry 
    FROM read_parquet('{path}')
    WHERE bbox.xmin >= {minx}
      AND bbox.xmax <= {maxx}
      AND bbox.ymin >= {miny}
      AND bbox.ymax <= {maxy}
    """
    arrow_table = conn.execute(query).fetch_arrow_table()
    df = arrow_table.to_pandas()
    gdf = gpd.GeoDataFrame(
        df,
        geometry=df['geometry'].apply(wkt.loads),
        crs="EPSG:4326"
    )
    return gdf

## Load London area

In [4]:
# London bbox based on example on https://wiki.openstreetmap.org/wiki/Bounding_box
# We could derive this based on the GERS id, but not for now
london_bbox = (-0.489,51.28,0.236,51.686)

In [5]:
overturemaps_base = "/Volumes/PRO-G40/OvertureMaps/data/release/2025-08-20.0"

In [6]:
conn = duckdb_connection()

In [12]:
london_segments_gdf = load_area(conn, overturemaps_base, "transportation", "segment", london_bbox)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
london_segments_gdf.head()

,id,bbox,version,sources,subtype,class,names,connectors,routes,subclass_rules,...,prohibited_transitions,road_surface,road_flags,speed_limits,width_rules,subclass,rail_flags,theme,type,geometry
0,48a00d70-128e-43bf-a25c-ca462c11a0fd,"{'xmin': -0.47854700684547424, 'xmax': -0.4778...",1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,track,None,[{'connector_id': 'df429e84-ef08-4021-b7e6-130...,None,None,...,None,None,None,None,None,None,None,transportation,segment,"LINESTRING (-0.47855 51.2814, -0.47819 51.2813..."
1,71fe3028-38f6-48a2-97cf-e06b59b2d8d1,"{'xmin': -0.46354350447654724, 'xmax': -0.4561...",1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,track,None,[{'connector_id': 'd8daf746-04a5-40cb-a2d4-4d4...,None,None,...,None,"[{'value': 'unpaved', 'between': None}]",None,None,None,None,None,transportation,segment,"LINESTRING (-0.46354 51.28084, -0.46261 51.281..."
2,573b990e-c2c8-43d5-8e60-a94fe9460c10,"{'xmin': -0.4549151062965393, 'xmax': -0.45467...",1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,service,None,[{'connector_id': 'c31f670d-bba6-403d-8d2f-60d...,None,"[{'value': 'parking_aisle', 'between': None}]",...,None,None,None,None,None,parking_aisle,None,transportation,segment,"LINESTRING (-0.45492 51.28084, -0.45468 51.280..."
3,d3fa2c07-7852-4acf-90af-720e6ed6d946,"{'xmin': -0.45137840509414673, 'xmax': -0.4513...",1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,footway,None,[{'connector_id': 'c8bca67f-b97b-4d36-a9ae-74f...,None,None,...,None,None,None,None,None,None,None,transportation,segment,"LINESTRING (-0.45138 51.28092, -0.4513 51.28077)"
4,018dabad-91ea-4a65-828d-2bb4f68d91f8,"{'xmin': -0.4553714096546173, 'xmax': -0.45446...",1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,service,None,[{'connector_id': '156f2ae5-8620-4837-833a-f1a...,None,None,...,None,None,None,None,None,None,None,transportation,segment,"LINESTRING (-0.45446 51.28175, -0.45537 51.2815)"
